In [2]:
import hyperopt as ho
import time
import pickle

In [3]:
ho.__file__

'/Users/angermue/python/venv/predict/lib/python3.4/site-packages/hyperopt-0.0.3.dev0-py3.4.egg/hyperopt/__init__.py'

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
def objective(params):
    x, y = params
    loss = (x - 5)**2 + y**2 + 2
    rv = {'loss': loss,
          'status': ho.STATUS_OK,
          'foo': 'bar', 
          'time': time.time()
          }   
    return rv

In [6]:
space = [ho.hp.uniform('x', -10, 10), ho.hp.uniform('y', -10, 10)]
best = ho.fmin(objective,
               space=space,
               algo=ho.rand.suggest,
               max_evals=100)
print(best)

{'x': 4.3641979844855587, 'y': -0.21327826234168157}


In [7]:
space = [ho.hp.uniform('x', -10, 10), ho.hp.uniform('y', -10, 10)]
best = ho.fmin(objective,
               space=space,
               algo=ho.tpe.suggest,
               max_evals=100)
print(best)

{'x': 4.9949027915052655, 'y': -0.49199850924305788}


In [9]:
trials = ho.Trials()
best = ho.fmin(objective,
               space=space,
               algo=ho.tpe.suggest,
               max_evals=5,
               trials=trials)

In [10]:
print(best)

{'x': 4.2441459757489763, 'y': 0.52597610769494985}


In [11]:
trials.argmin

{'x': 4.2441459757489763, 'y': 0.52597610769494985}

In [12]:
trials.results

[{'foo': 'bar',
  'loss': 25.034355717177647,
  'status': 'ok',
  'time': 1436637224.356363},
 {'foo': 'bar',
  'loss': 2.8479661718423968,
  'status': 'ok',
  'time': 1436637224.357794},
 {'foo': 'bar',
  'loss': 26.731884532581397,
  'status': 'ok',
  'time': 1436637224.359201},
 {'foo': 'bar',
  'loss': 101.52803932698217,
  'status': 'ok',
  'time': 1436637224.360644},
 {'foo': 'bar',
  'loss': 51.879137915279856,
  'status': 'ok',
  'time': 1436637224.36213}]

In [13]:
trials.vals

{'x': [5.4445365220551238,
  4.2441459757489763,
  1.5722848767502882,
  -2.0820120428574214,
  0.86363773047252401],
 'y': [-4.7787804927341853,
  0.52597610769494985,
  3.6031449549562407,
  7.0266026465002724,
  5.7244777133385156]}

In [14]:
trials.losses()

[25.034355717177647,
 2.8479661718423968,
 26.731884532581397,
 101.52803932698217,
 51.879137915279856]

In [16]:
trials.statuses()

['ok', 'ok', 'ok', 'ok', 'ok']

In [17]:
trials.trials

[{'book_time': datetime.datetime(2015, 7, 11, 17, 53, 44, 356000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'x': [0], 'y': [0]},
   'tid': 0,
   'vals': {'x': [5.4445365220551238], 'y': [-4.7787804927341853]},
   'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2015, 7, 11, 17, 53, 44, 356000),
  'result': {'foo': 'bar',
   'loss': 25.034355717177647,
   'status': 'ok',
   'time': 1436637224.356363},
  'spec': None,
  'state': 2,
  'tid': 0,
  'version': 0},
 {'book_time': datetime.datetime(2015, 7, 11, 17, 53, 44, 357000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'x': [1], 'y': [1]},
   'tid': 1,
   'vals': {'x': [4.2441459757489763], 'y': [0.52597610769494985]},
   'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2015, 7, 11, 17, 53, 44, 357000),
  'result': {'foo': 'bar',
   'loss': 2.8479661718423968,
   'status': 'ok',
   'time': 143663722

In [18]:
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as met

In [19]:
digits = ds.load_digits()

In [20]:
X = digits.data
y = digits.target == 8
n = X.shape[0]
val = 0.2
nval = int(n * val)
val_X = X[:nval]
val_y = y[:nval]
train_X = X[nval:]
train_y = y[nval:]

In [21]:
ne = []

def objective(p):
    #m = RandomForestClassifier(
    #    n_estimators=1+p['n_estimators'], 
    #    max_depth=round(p['max_depth']), 
    #    criterion=p['criterion'])
    m = RandomForestClassifier()
    ne.append(p['max_depth'])
    
    m.set_params(**p)
    m.fit(train_X, train_y)
    z = m.predict(train_X)
    train_acc = met.accuracy_score(train_y, z)
    train_auc = met.roc_auc_score(train_y, z)
    z = m.predict(val_X)
    val_acc = met.accuracy_score(val_y, z)
    val_auc = met.roc_auc_score(val_y, z)
    rv = {'loss': 1-val_acc, 'auc': val_auc, 'train_auc': train_auc, 'status': ho.STATUS_OK}
    return rv

In [22]:
trials = ho.Trials()
space = {'n_estimators': 1 + ho.hp.randint('n_estimators', 15), 
         'max_depth': ho.hp.quniform('max_depth', 1, 20, 1),
         'criterion': ho.hp.choice('criterion', ['gini', 'entropy'])
        }
best = ho.fmin(objective,
               space=space,
               algo=ho.tpe.suggest,
               max_evals=10, trials=trials)

In [23]:
best

{'criterion': 1, 'max_depth': 20.0, 'n_estimators': 6}

In [33]:
def my_fun(a=10, b=20):
    return a + b

In [38]:
m = RandomForestClassifier()
p = {'n_estimators': 1}
m.set_params(**p)
m.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [60]:
space = {'n_estimators': 1 + ho.hp.randint('n_estimators', 15), 
         'max_depth': ho.hp.quniform('max_depth', 1, 20, 1),
         'criterion': ho.hp.choice('criterion', ['gini', 'entropy'])
        }

In [64]:
ho.pyll.stochastic.sample(space)

{'criterion': 'gini', 'max_depth': 4.0, 'n_estimators': 10}

In [66]:
space = [ho.hp.choice('nhidden', [0, 1, 2]), 1 + ho.hp.randint('nunits1', 10), 1 + ho.hp.randint('nunits2', 10)]

In [67]:
ho.pyll.stochastic.sample(space)

(1, 2, 2)

In [ ]:
ho.H

In [34]:
d = {'a': 10, 'b': 20}

In [35]:
my_fun(**d)

30

In [141]:
trials.results

[{'auc': 0.87312217194570141,
  'loss': 0.03899721448467963,
  'status': 'ok',
  'train_auc': 0.97499999999999998},
 {'auc': 0.87312217194570141,
  'loss': 0.03899721448467963,
  'status': 'ok',
  'train_auc': 0.97857142857142865},
 {'auc': 0.80266968325791854,
  'loss': 0.0473537604456824,
  'status': 'ok',
  'train_auc': 0.96032907770195897},
 {'auc': 0.84832579185520363,
  'loss': 0.03621169916434541,
  'status': 'ok',
  'train_auc': 0.99285714285714288},
 {'auc': 0.91877828054298649,
  'loss': 0.027855153203342642,
  'status': 'ok',
  'train_auc': 0.91516068677085627}]